# Notebook 5: Full Analysis for Thesis

This notebook generates all artifacts needed for thesis presentation:
- Comparison grids showing pipeline stages
- Performance benchmarks
- Statistical analysis
- Publication-ready figures

## System Overview
- **Detection**: YOLO-World (zero-shot, custom classes)
- **Depth**: Depth Anything V2 (monocular)
- **Fusion**: Priority-based obstacle ranking

In [ ]:
import sys
sys.path.insert(0, '..')

import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import time
from datetime import datetime
from collections import Counter
import json

from src.config import Config
from src.detector import Detector
from src.depth import DepthEstimator
from src.fusion import FusionEngine
from src.utils import Timer, create_side_by_side

plt.rcParams['figure.dpi'] = 150
plt.rcParams['savefig.dpi'] = 300
print("Analysis notebook ready!")

## Setup

In [ ]:
from src.config import DetectionConfig, DepthConfig, FusionConfig

# YOLO-World with custom navigation classes
detection_config = DetectionConfig(
    model="yolov8s-world.pt",
    confidence=0.2,
    classes=[
        "door", "person", "chair", "table", "stairs",
        "wall", "window", "car", "bicycle", "obstacle"
    ]
)

depth_config = DepthConfig(model="vits")
fusion_config = FusionConfig(danger_zone=1.5, warning_zone=3.0)

detector = Detector(detection_config)
depth_estimator = DepthEstimator(depth_config)
fusion = FusionEngine(fusion_config)

print("Loading YOLO-World...")
detector.load()
print(f"  Classes: {detector.class_names}")

print("Loading Depth Anything V2...")
depth_estimator.load()
print("Models loaded!")

captures_dir = Path("../data/captures")
samples_dir = Path("../data/sample_images")
results_dir = Path("../data/results")
results_dir.mkdir(parents=True, exist_ok=True)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

## Load All Images

In [ ]:
image_files = list(captures_dir.glob("*.jpg")) + list(samples_dir.glob("*.jpg"))
print(f"Found {len(image_files)} images for analysis")

images = []
for path in image_files:
    img = cv2.imread(str(path))
    if img is not None:
        images.append({'path': path, 'frame': img, 'name': path.stem})

print(f"Loaded {len(images)} images successfully")

## Process All Images and Collect Metrics

In [ ]:
results = []
detection_times = []
depth_times = []
fusion_times = []

print("Processing images...")
for i, img_data in enumerate(images):
    frame = img_data['frame']
    name = img_data['name']
    
    t0 = time.perf_counter()
    detections = detector.detect(frame)
    t1 = time.perf_counter()
    depth_map = depth_estimator.estimate(frame)
    t2 = time.perf_counter()
    obstacles = fusion.process(detections, depth_map, frame.shape[1])
    t3 = time.perf_counter()
    
    det_time = (t1 - t0) * 1000
    dep_time = (t2 - t1) * 1000
    fus_time = (t3 - t2) * 1000
    
    detection_times.append(det_time)
    depth_times.append(dep_time)
    fusion_times.append(fus_time)
    
    results.append({
        'name': name,
        'frame': frame,
        'detections': detections,
        'depth_map': depth_map,
        'obstacles': obstacles,
        'det_time': det_time,
        'dep_time': dep_time,
        'fus_time': fus_time,
        'total_time': det_time + dep_time + fus_time
    })
    
    if (i + 1) % 5 == 0:
        print(f"  Processed {i+1}/{len(images)}")

print(f"\nProcessing complete!")

## Performance Analysis

In [ ]:
print("=" * 60)
print("PERFORMANCE ANALYSIS")
print("=" * 60)

total_times = [r['total_time'] for r in results]

print(f"\nDetection (YOLO-World):")
print(f"  Mean: {np.mean(detection_times):.1f}ms")
print(f"  Std:  {np.std(detection_times):.1f}ms")
print(f"  Min:  {np.min(detection_times):.1f}ms")
print(f"  Max:  {np.max(detection_times):.1f}ms")

print(f"\nDepth Estimation (Depth Anything V2 - CPU):")
print(f"  Mean: {np.mean(depth_times):.1f}ms")
print(f"  Std:  {np.std(depth_times):.1f}ms")
print(f"  Min:  {np.min(depth_times):.1f}ms")
print(f"  Max:  {np.max(depth_times):.1f}ms")

print(f"\nFusion:")
print(f"  Mean: {np.mean(fusion_times):.2f}ms")

print(f"\nTotal Pipeline:")
print(f"  Mean: {np.mean(total_times):.1f}ms")
print(f"  FPS:  {1000/np.mean(total_times):.2f}")

## Performance Visualization

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

categories = ['YOLO-World', 'Depth', 'Fusion']
means = [np.mean(detection_times), np.mean(depth_times), np.mean(fusion_times)]
stds = [np.std(detection_times), np.std(depth_times), np.std(fusion_times)]

bars = axes[0].bar(categories, means, yerr=stds, capsize=5, 
                   color=['steelblue', 'coral', 'seagreen'])
axes[0].set_ylabel('Time (ms)')
axes[0].set_title('Processing Time by Stage')

axes[1].pie(means, labels=categories, autopct='%1.1f%%',
            colors=['steelblue', 'coral', 'seagreen'])
axes[1].set_title('Time Distribution')

axes[2].hist(total_times, bins=15, color='steelblue', edgecolor='white')
axes[2].axvline(np.mean(total_times), color='red', linestyle='--', 
                label=f'Mean: {np.mean(total_times):.1f}ms')
axes[2].set_xlabel('Total Time (ms)')
axes[2].set_ylabel('Count')
axes[2].set_title('Total Pipeline Time Distribution')
axes[2].legend()

plt.tight_layout()
plt.savefig(results_dir / f"performance_analysis_{timestamp}.png")
plt.show()

## Detection Statistics

In [ ]:
all_detections = []
all_obstacles = []

for r in results:
    all_detections.extend(r['detections'])
    all_obstacles.extend(r['obstacles'])

print("=" * 60)
print("DETECTION STATISTICS")
print("=" * 60)

print(f"\nTotal detections: {len(all_detections)}")
print(f"Total obstacles: {len(all_obstacles)}")
print(f"Average detections per image: {len(all_detections)/len(results):.1f}")

if all_detections:
    class_counts = Counter([d.class_name for d in all_detections])
    confidences = [d.confidence for d in all_detections]
    
    print(f"\nConfidence: {np.mean(confidences):.2f} +/- {np.std(confidences):.2f}")
    print(f"\nTop 10 Classes:")
    for cls, count in class_counts.most_common(10):
        print(f"  {cls}: {count}")

## Obstacle Analysis

In [ ]:
if all_obstacles:
    distances = [o.distance for o in all_obstacles]
    positions = Counter([o.position for o in all_obstacles])
    
    danger = sum(1 for o in all_obstacles if o.is_danger)
    warning = sum(1 for o in all_obstacles if o.is_warning and not o.is_danger)
    safe = len(all_obstacles) - danger - warning
    
    print("\n" + "=" * 60)
    print("OBSTACLE ANALYSIS")
    print("=" * 60)
    
    print(f"\nDistance Statistics:")
    print(f"  Mean: {np.mean(distances):.2f}m")
    print(f"  Min:  {np.min(distances):.2f}m")
    print(f"  Max:  {np.max(distances):.2f}m")
    
    print(f"\nZone Distribution:")
    print(f"  Danger (<1.5m):  {danger} ({100*danger/len(all_obstacles):.1f}%)")
    print(f"  Warning (1.5-3m): {warning} ({100*warning/len(all_obstacles):.1f}%)")
    print(f"  Safe (>3m):       {safe} ({100*safe/len(all_obstacles):.1f}%)")
    
    print(f"\nPosition Distribution:")
    for pos, count in positions.items():
        print(f"  {pos}: {count} ({100*count/len(all_obstacles):.1f}%)")

## Generate Comparison Grid

In [ ]:
n_samples = min(4, len(results))
samples = results[:n_samples]

fig, axes = plt.subplots(n_samples, 4, figsize=(16, 4*n_samples))
if n_samples == 1:
    axes = axes.reshape(1, -1)

for i, r in enumerate(samples):
    frame = r['frame']
    detections = r['detections']
    depth_map = r['depth_map']
    obstacles = r['obstacles']
    
    axes[i, 0].imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    axes[i, 0].set_title(f"Original: {r['name']}")
    axes[i, 0].axis('off')
    
    det_frame = detector.draw_detections(frame.copy(), detections)
    axes[i, 1].imshow(cv2.cvtColor(det_frame, cv2.COLOR_BGR2RGB))
    axes[i, 1].set_title(f"Detection ({len(detections)} objects)")
    axes[i, 1].axis('off')
    
    depth_colored = depth_estimator.colorize(depth_map)
    axes[i, 2].imshow(cv2.cvtColor(depth_colored, cv2.COLOR_BGR2RGB))
    axes[i, 2].set_title("Depth Map")
    axes[i, 2].axis('off')
    
    fus_frame = fusion.draw_obstacles(frame.copy(), obstacles)
    axes[i, 3].imshow(cv2.cvtColor(fus_frame, cv2.COLOR_BGR2RGB))
    axes[i, 3].set_title(f"Fusion ({len(obstacles)} obstacles)")
    axes[i, 3].axis('off')

plt.suptitle("Smart Aid Pipeline Comparison", fontsize=16, y=1.02)
plt.tight_layout()
plt.savefig(results_dir / f"comparison_grid_{timestamp}.png", bbox_inches='tight')
plt.show()

## Export Results Summary

In [ ]:
summary = {
    'timestamp': timestamp,
    'num_images': len(results),
    'performance': {
        'detection_ms': {
            'mean': float(np.mean(detection_times)),
            'std': float(np.std(detection_times))
        },
        'depth_ms': {
            'mean': float(np.mean(depth_times)),
            'std': float(np.std(depth_times))
        },
        'total_ms': float(np.mean(total_times)),
        'fps': float(1000/np.mean(total_times))
    },
    'detections': {
        'total': len(all_detections),
        'classes': dict(Counter([d.class_name for d in all_detections]).most_common(10))
    },
    'obstacles': {
        'total': len(all_obstacles),
        'danger_count': sum(1 for o in all_obstacles if o.is_danger),
        'warning_count': sum(1 for o in all_obstacles if o.is_warning and not o.is_danger),
        'avg_distance': float(np.mean([o.distance for o in all_obstacles])) if all_obstacles else 0
    }
}

summary_path = results_dir / f"analysis_summary_{timestamp}.json"
with open(summary_path, 'w') as f:
    json.dump(summary, f, indent=2)

print(f"Summary saved to: {summary_path}")

## Thesis-Ready Summary Table

In [ ]:
print("\n" + "=" * 60)
print("THESIS SUMMARY TABLE")
print("=" * 60)
print(f"""
| Metric                    | Value            |
|---------------------------|------------------|
| Images Processed          | {len(results)}              |
| Total Detections          | {len(all_detections)}              |
| Total Obstacles           | {len(all_obstacles)}              |
| Detection Time (mean)     | {np.mean(detection_times):.1f} ms          |
| Depth Time (mean)         | {np.mean(depth_times):.1f} ms          |
| Total Pipeline (mean)     | {np.mean(total_times):.1f} ms          |
| Effective FPS             | {1000/np.mean(total_times):.1f}              |
| Danger Zone Detections    | {sum(1 for o in all_obstacles if o.is_danger)} ({100*sum(1 for o in all_obstacles if o.is_danger)/max(1,len(all_obstacles)):.1f}%)         |
""")

## Summary

This analysis notebook has generated:

1. **Performance metrics** - Timing for each pipeline stage
2. **Detection statistics** - Class distribution, confidence scores
3. **Obstacle analysis** - Distance zones, positions
4. **Visual comparisons** - Grid showing pipeline stages
5. **JSON summary** - Machine-readable results

All outputs are saved in `data/results/` for thesis inclusion.

## Key Findings

| Aspect | Standard YOLO | Our System (YOLO-World) |
|--------|---------------|------------------------|
| Door detection | No | Yes |
| Stairs detection | No | Yes |
| Custom classes | Limited to 80 | Unlimited |
| Depth estimation | Requires extra sensor | Single RGB camera |
| Cost | N/A | ~$250 |